In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objs as go

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


### Start with common stock price

In [2]:
stock = "NVDA"
ticker = yf.Ticker(stock)
ticker_data = ticker.history()


In [3]:
candlestick = go.Candlestick(
    x=ticker_data.index,
    open=ticker_data["Open"],
    high=ticker_data["High"],
    low=ticker_data["Low"],
    close=ticker_data["Close"],
    name="Candles"
    )

fig = go.Figure(data=[candlestick])
fig.update_layout(title=f"{stock} stock price",
                  xaxis_title="Date",
                  yaxis_title="Price",
                  xaxis_rangeslider_visible=False)
fig.show()

### Setup for options

In [4]:
close_price = ticker_data["Close"][-1]
print(close_price)

903.6300048828125


In [5]:
op_date = ticker.options
op_date[:10]

('2024-04-05',
 '2024-04-12',
 '2024-04-19',
 '2024-04-26',
 '2024-05-03',
 '2024-05-10',
 '2024-05-17',
 '2024-06-21',
 '2024-07-19',
 '2024-08-16')

In [6]:
call_chain = ticker.option_chain(op_date[0]).calls
atm_idx = (call_chain['strike']-close_price).abs().idxmin()

atm_price = call_chain.iloc[atm_idx-2:atm_idx+2].nlargest(1, 'openInterest')["strike"].values[0]
itm_prices = call_chain.iloc[atm_idx-20:atm_idx-10].nlargest(2, 'openInterest')["strike"].values
otm_prices = call_chain.iloc[atm_idx+10:atm_idx+20].nlargest(2, 'openInterest')["strike"].values

In [7]:
strike_prices = np.append(otm_prices, itm_prices)
strike_prices = np.append(strike_prices, atm_price)
strike_prices.sort()

In [8]:
print(strike_prices)

[830. 870. 900. 940. 950.]


In [9]:
def fixed_strike_diff_expire(strike_price):
    normalised = []
    op_price = []
    ratio = []

    return_ratio = pd.DataFrame()

    for i in [0,2,4,6,8]:
        try:
            op_call = ticker.option_chain(op_date[i]).calls

            op_call_symbol = op_call[op_call["strike"] == strike_price]["contractSymbol"].values[0]

            op_call_hist = yf.Ticker(op_call_symbol)
            op_call_price = op_call_hist.history()

            # Get the change % daily for normalized comparison between diff op price 
            op_call_price["Close_diff"] = op_call_price["Close"].pct_change() * 100

            return_ratio[op_date[i]] = abs(op_call_price["Close_diff"])

            given_date = datetime.strptime(op_date[i], "%Y-%m-%d")
            today_date = datetime.now()
            days_remain = (given_date - today_date).days

            normalised.append(go.Scatter(x=op_call_price.index, 
                                        y=op_call_price["Close_diff"], 
                                        mode="lines+markers",
                                        name=f"{op_date[i]}, in {days_remain}")
                                        )
            op_price.append(go.Scatter(x=op_call_price.index, 
                                    y=op_call_price["Close"], 
                                    mode="lines+markers", 
                                    name=f"{op_date[i]}, in {days_remain}")
                                    )
        except:
            pass
        
    normed_ratio = return_ratio.div(return_ratio.min(axis=1), axis=0)
    for col in normed_ratio.columns:
        ratio.append(go.Scatter(x=normed_ratio.index,
                                y=normed_ratio[col],
                                mode="markers+lines",
                                name=col))

    fig1 = go.Figure(data=normalised)
    fig1.update_layout(title=f"{stock} Call Daily return (strike price = {strike_price})",
                       xaxis_title="Date",
                       yaxis_title="Daily Returns (%)")
    fig1.show()

    fig2 = go.Figure(data=op_price)
    fig2.update_layout(title=f"{stock} Call price (strike price = {strike_price})",
                       xaxis_title="Date",
                       yaxis_title="OP price (USD)")
    fig2.show()

    fig3 = go.Figure(data=ratio)
    fig3.update_layout(title=f"{stock} Call daily return ratio (strike price = {strike_price})",
                       xaxis_title="Date",
                       yaxis_range=[0, 6],
                       yaxis_title="ratio (times)")
    fig3.show()

    return 

In [10]:
def fixed_expire_diff_strike(expire, strike_prices):
    normalised = []
    op_price = []

    op_call = ticker.option_chain(expire).calls

    for strike_price in strike_prices:
        try:
            op_call_symbol = op_call[op_call["strike"] == strike_price]["contractSymbol"].values[0]

            op_call_hist = yf.Ticker(op_call_symbol)
            op_call_price = op_call_hist.history()

            # Get the change % daily for normalized comparison between diff op price 
            op_call_price["Close_diff"] = op_call_price["Close"].pct_change() * 100

            normalised.append(go.Scatter(x=op_call_price.index, 
                                        y=op_call_price["Close_diff"], 
                                        mode="lines+markers",
                                        name=str(strike_price))
                                        )
            op_price.append(go.Scatter(x=op_call_price.index, 
                                    y=op_call_price["Close"], 
                                    mode="lines+markers", 
                                    name=str(strike_price))
                                    )
        except:
            pass
        
    given_date = datetime.strptime(expire, "%Y-%m-%d")
    today_date = datetime.now()
    days_remain = (given_date - today_date).days

    fig1 = go.Figure(data=normalised)
    fig1.update_layout(title=f"{stock} Call Daily return (expire date = {expire}, in {days_remain} days)",
                       xaxis_title="Date",
                       yaxis_title="Daily Returns (%)")
    fig1.show()

    fig2 = go.Figure(data=op_price)
    fig2.update_layout(title=f"{stock} Call price (expire date = {expire}, in {days_remain} days)",
                       xaxis_title="Date",
                       yaxis_title="OP price (USD)")
    fig2.show()

    return

In [11]:
def sample_option_price(factor):
    return 1 * (1+(0.90*factor)) * (1+(-0.1*factor)) * (1+(-0.2*factor)) * (1+(-0.2*factor)) * (1+(0.3*factor))

for i in [0.5, 0.8, 1.0, 1.3, 1.6, 2.0, 4.0]:
    print(f"{i} : {sample_option_price(i)}")


0.5 : 1.28314125
0.8 : 1.3845113856
1.0 : 1.4227200000000002
1.3 : 1.4370015156000002
1.6 : 1.4026478592
2.0 : 1.2902399999999998
4.0 : 0.2428799999999999


### Compare (1/6): Diff exp time for same strike price (in the money)

In [12]:
fixed_strike_diff_expire(strike_price=itm_prices[0])

### Compare (2/6): Diff exp time for same strike price (at the money)

In [13]:
fixed_strike_diff_expire(strike_price=atm_price)

### Compare (3/6): Diff exp time for same strike price (out the money)

In [14]:
fixed_strike_diff_expire(strike_price=otm_prices[0])

### Compare (4/6): Same exp time for diff strike price (<2 weeks)

In [15]:
expire = op_date[1]
fixed_expire_diff_strike(expire, strike_prices)

### Compare (5/6): Same exp time for diff strike price (~30 days)

In [16]:
expire = op_date[3]
fixed_expire_diff_strike(expire, strike_prices)

### Compare (6/6): Same exp time for diff strike price (>30 days)

In [17]:
expire = op_date[4]
fixed_expire_diff_strike(expire, strike_prices)

In [18]:
expire = op_date[6]
fixed_expire_diff_strike(expire, strike_prices)

In [19]:
expire = op_date[8]
fixed_expire_diff_strike(expire, strike_prices)

In [20]:
expire = op_date[9]
fixed_expire_diff_strike(expire, strike_prices)

In [21]:
expire = op_date[10]
fixed_expire_diff_strike(expire, strike_prices)
